In [1]:
# data manipulation
import pandas as pd
import numpy as np
import itertools

# explore
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# nlp
import unicodedata
import re
import nltk 
from nltk.corpus import stopwords
import nltk.sentiment
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# modeling
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import naive_bayes as nb

from sklearn.model_selection import train_test_split

# local
import wrangle as w

imports loaded successfully, awaiting commands...


In [2]:
# from google.oauth2 import service_account
# import pandas_gbq

# credentials = service_account.Credentials.from_service_account_file(
#     'my-ds-projects-d864a770b51b.json',
# )
# df = pandas_gbq.read_gbq("select * from cfpb_complaints.complaint_database",dialect='standard',project_id="my-ds-projects",use_bqstorage_api=True,credentials=credentials)
# df

In [3]:
df = w.check_file_exists_gbq('cfpb.csv','service_key.json')
df

csv file found and loaded


,date_received,product,subproduct,issue,subissue,consumer_complaint_narrative,company_public_response,company_name,state,zip_code,tags,consumer_consent_provided,submitted_via,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed,complaint_id
0,2022-04-11,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,Hello my name is XXXX XXXX XXXX and I am submi...,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,MD,20720,NaN,Consent provided,Web,2022-04-11,Closed with explanation,True,NaN,5429978
1,2022-11-17,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,NaN,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,TX,78577.0,NaN,Consent not provided,Web,2022-11-17,Closed with explanation,True,NaN,6210155
2,2020-05-27,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,NaN,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,SC,29229,NaN,Consent not provided,Web,2020-05-27,Closed with explanation,True,NaN,3670764
3,2020-03-28,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,I have review my credit report to check accura...,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,FL,32825.0,Servicemember,Consent provided,Web,2020-03-28,Closed with non-monetary relief,True,NaN,3584127
4,2022-06-22,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,I recently noticed that my credit report conta...,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,OH,43219,NaN,Consent provided,Web,2022-06-22,Closed with explanation,True,NaN,5697586
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3458901,2023-03-06,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Account status incorrect,NaN,NaN,"Colony Brands, Inc.",WI,53223.0,NaN,NaN,Web,2023-03-06,Closed with explanation,True,NaN,6655295
3458902,2022-03-23,Checking or savings account,Checking account,Opening an account,Unable to open an account,"I contracted XXXX was hospitalized, I there we...",Company has responded to the consumer and the ...,"Credit Control, LLC",GA,30078.0,NaN,Consent provided,Web,2022-03-23,Closed with explanation,True,NaN,5358651
3458903,2014-11-23,Debt collection,Credit card,Taking/threatening an illegal action,Threatened to sue on too old debt,NaN,NaN,"Hiday & Ricke, P.A.",AR,85747,NaN,NaN,Web,2014-11-23,Closed with explanation,True,True,1130481
3458904,2022-04-19,Credit card or prepaid card,Store credit card,Getting a credit card,Application denied,"I recently applied for XXXX XXXX credit card, ...",Company has responded to the consumer and the ...,SYNCHRONY FINANCIAL,MD,20746.0,Older American,Consent provided,Web,2022-04-19,Closed with explanation,True,NaN,5464598


In [4]:
df_clean = w.clean_data(df.copy())
df_clean.head()

,date_received,narrative,company_name,state,tags,company_response_to_consumer,product_bins
0,2022-04-11,Hello my name is XXXX XXXX XXXX and I am submi...,Experian Information Solutions Inc.,MD,Average Person,Closed with explanation,credit_report
1,2020-03-28,I have review my credit report to check accura...,Experian Information Solutions Inc.,FL,Servicemember,Closed with non-monetary relief,credit_report
2,2022-06-22,I recently noticed that my credit report conta...,Experian Information Solutions Inc.,OH,Average Person,Closed with explanation,credit_report
3,2018-04-19,I am a victim of identity theft and been a vic...,Experian Information Solutions Inc.,AZ,Servicemember,Closed with non-monetary relief,credit_report
4,2021-10-11,Company has not responded to my request for in...,Experian Information Solutions Inc.,TX,Average Person,Closed with explanation,credit_report


In [5]:
pd.set_option('display.max_colwidth',None)

In [23]:
def prep_narrative(df):
    df = df.assign(no_x = df.apply(lambda row : re.sub(r'[X{1,}\d\']', '', row.narrative), axis=1))
    # Derive column 'clean' from column: cleanup up 'readme_contents'
    df = df.assign(clean = df.apply(lambda row : w.remove_stopwords(w.token_it_up(w.basic_clean(row.no_x))), axis=1))
    # Derive column 'lemmatized' from column: lemmatized 'clean'
    df = df.assign(lemon = df.apply(lambda row : w.lemmad(row.clean), axis=1))
    # return prepped df
    return df

In [24]:
df_clean_sample = prep_narrative(df_clean.sample(1000))
df_clean_sample

,date_received,narrative,company_name,state,tags,company_response_to_consumer,product_bins,no_x,clean,lemon
453641,2021-09-20,"JUNK DEBT BUYER IN VIOLATION OF THE FDCPA - IN VIOLATION OF MY FEDERALLY PROTECTED CONSUMER RIGHTS- I AM THE CONSUMER- I AM THE ADMINISTRATOR- I AM THE EXECUTOR 15 USC 1692c ( d ) Ability Recovery Services claims that their client XXXX XXXX XXXX ( who they "" represent '' ) has bought this debt, so a person would not owe this alleged debt to Ability Recovery Services, their client or the assumed original creditor ; Ability Recovery Services just admitted to False or Misleading Representations pursuant to 15 USC 1692e ( 6 ) ( A ). Also, I, the federally protected consumer, never gave them permission to have my personal information, that is a violation of my right to privacy pursuant to 15 USC 1692a. Furthermore, Ability Recovery Services just admitted to participating in conspiring against my rights with their client, pursuant to 18 USC 241, which they shall be fined and imprisoned for 10 years. They are violating 18 USC 1028, by unlawfully transferring my authentication feature ( social security number ) electronically. This debt collector is violating a number of my FEDERALLY PROTECTED CONSUMER RIGHTS ; some, but not all, are included in my Bill of Particulars. Delete this alleged debt from all of my consumer reports immediately and terminate all further collection efforts.","Ability Recovery Services, LLC",IL,Average Person,Closed with explanation,debt_collection,"JUNK DEBT BUYER IN VIOLATION OF THE FDCPA - IN VIOLATION OF MY FEDERALLY PROTECTED CONSUMER RIGHTS- I AM THE CONSUMER- I AM THE ADMINISTRATOR- I AM THE EECUTOR USC c ( d ) Ability Recovery Services claims that their client ( who they "" represent ) has bought this debt so a person would not owe this alleged debt to Ability Recovery Services their client or the assumed original creditor ; Ability Recovery Services just admitted to False or Misleading Representations pursuant to USC e ( ) ( A ). Also I the federally protected consumer never gave them permission to have my personal information that is a violation of my right to privacy pursuant to USC a. Furthermore Ability Recovery Services just admitted to participating in conspiring against my rights with their client pursuant to USC which they shall be fined and imprisoned for years. They are violating USC by unlawfully transferring my authentication feature ( social security number ) electronically. This debt collector is violating a number of my FEDERALLY PROTECTED CONSUMER RIGHTS ; some but not all are included in my Bill of Particulars. Delete this alleged debt from all of my consumer reports immediately and terminate all further collection efforts.",junk debt buyer violation fdcpa violation federally protected consumer rights consumer administrator eecutor usc c ability recovery services claims client represent bought debt person would owe alleged debt ability recovery services client assumed original creditor ability recovery services admitted false misleading representations pursuant usc e also federally protected consumer never gave permission personal information violation right privacy pursuant usc furthermore ability recovery services admitted participating conspiring rights client pursuant usc shall fined imprisoned years violating usc unlawfully transferring authentication feature social security number electronically debt collector violating number federally protected consumer rights included bill particulars delete alleged debt consumer reports immediately terminate collection efforts,junk debt buyer violation fdcpa violation federally protected consumer right consumer administrator eecutor usc c ability recovery service claim client represent bought debt person would owe alleged debt ability recovery service client assumed original creditor ability recovery service admitted false misleading representation pursuant usc e also federally protected consumer never gave permissio

In [25]:
df_clean.head(3)

,date_received,narrative,company_name,state,tags,company_response_to_consumer,product_bins
0,2022-04-11,Hello my name is XXXX XXXX XXXX and I am submitting this complaint without error or influence of any third agency. Please see my attached documentation as it contains my request.,Experian Information Solutions Inc.,MD,Average Person,Closed with explanation,credit_report
1,2020-03-28,"I have review my credit report to check accuracy and found out that a lot of account that I had no knowledge about are destroying my credit reputation and causing a lot of damage to my ability to obtain credit, these account are fraudulent and should not be in my credit, I what to request that these account be block from my credit immediately. \nThank you.",Experian Information Solutions Inc.,FL,Servicemember,Closed with non-monetary relief,credit_report
2,2022-06-22,"I recently noticed that my credit report contained what looks like a car from XXXX XXXX. This is not my car, period. I also tried reaching out to XXXX - Experian, XXXX and XXXX- but never heard back from any of them. So, at this point I'm not sure whatelse to do - Therefore - I recently filled a police report, which I am including as well.",Experian Information Solutions Inc.,OH,Average Person,Closed with explanation,credit_report


In [26]:
# df_clean['narrative'][0].apply(lambda row : re.sub(r'[X{1,}]', '', row.narrative), axis=1)
re.sub(r'[X{1,}\d\']', '', df_clean['narrative'][988513])

'Not sure where to begin ... Please look at previous complaints against Wells Fargo and Debt Buyers : CFPB Case Numbers   and . These complaints were submitted in  of  while in the middle of my court battle with the Debt Buyer. Wells Fargo really did a number scamming me out of my vehicle AND I was a service member. I just hope that in the limelight of the recent scandal and settlement that those that have been affected so much are truly compensated. Not to mention that during all of this I was completely unaware of the fact that when I financed the van through Wells Fargo in  that they put my interest rate @  % which they should nt have went over  %. I was a service member at this time and should not have had to go through what they put me through or my wife. I finally gave up in court and settled with the Debt Buyer. I feel that Wells Fargo should compensate me for all the money and distress I went through as a result of their negligence. I have  of court paperwork. They need to be a

In [43]:
df_clean_sample[['narrative','clean','lemon']].sample(1)
# df_clean_sample['narrative'][2613147]

,narrative,clean,lemon
295937,"I sent XXXX letters to Experian ( XXXX in XXXX and XXXX in XXXX, 30 day apart ) concerning inaccurate information on my report concerning bankruptcy. I ask Experian to provide me with copies of any documentation associated with this account, bearing my signature. In the absence of any such documentation bearing my signature, I formally request that this information be immediately deleted from the credit file you maintain under my Social Security number. I received a letter on XXXX XXXX, 2016 stating that they have previously processed this dispute and the credit grantor has verified its accuracy. Last but not least, they said that unless you send us additional relevant information to support your claim, WE WILL NOT REINVESTIGATE THIS INFORMATION AGAIN. EXPERIAN has not providing me with copies of documentation bearing my signature in a timely manner. Thank you for your help.",sent letters experian day apart concerning inaccurate information report concerning bankruptcy ask experian provide copies documentation associated account bearing signature absence documentation bearing signature formally request information immediately deleted credit file maintain social security number received letter stating previously processed dispute credit grantor verified accuracy last least said unless send us additional relevant information support claim reinvestigate information eperian providing copies documentation bearing signature timely manner thank help,sent letter experian day apart concerning inaccurate information report concerning bankruptcy ask experian provide copy documentation associated account bearing signature absence documentation bearing signature formally request information immediately deleted credit file maintain social security number received letter stating previously processed dispute credit grantor verified accuracy last least said unless send u additional relevant information support claim reinvestigate information eperian providing copy documentation bearing signature timely manner thank help
